This notebook will pull from the FPL API the weekly results for each player and stack all the results into a single df

Update weekly

In [76]:
import pandas as pd
import numpy as np
import requests

In [77]:
top500 = pd.read_csv('top500.csv', index_col='id_')
teams = pd.read_csv('teams', index_col=0)

In [93]:
top500.iloc[484]

assists                                  0
bonus                                    0
bps                                      0
chance_of_playing_next_round           NaN
chance_of_playing_this_round           NaN
clean_sheets                             0
code                                223349
cost_change_event                        0
cost_change_event_fall                   0
cost_change_start                        0
cost_change_start_fall                   0
creativity                               0
dreamteam_count                          0
element_type                             3
ep_next                                0.7
ep_this                               -0.3
event_points                             0
first_name                         Indiana
form                                     0
goals_conceded                           0
goals_scored                             0
ict_index                                0
in_dreamteam                         False
influence  

In [79]:
histories = {}

for id_ in top500.index:
    url = 'https://fantasy.premierleague.com/api/element-summary/' + str(id_) + '/'
    r = requests.get(url)
    player_data = r.json()
    player_history = player_data['history']
    
    histories[id_] = player_history

In [80]:
salah = pd.DataFrame(histories[191])

In [81]:
salah['total_points']

0     12
1      3
2     15
3      6
4      9
5      5
6      3
7      2
8      0
9      7
10     2
11     8
12     0
13     3
14     0
15    13
16    16
17     3
18     3
19    10
20     6
Name: total_points, dtype: int64

In [82]:
columns_list = []
for key in histories:
    df = pd.DataFrame(histories[key])
    columns_list.append(df.columns)
        

In [83]:
len(columns_list)

500

In [84]:
columns_list[499]

Index(['assists', 'bonus', 'bps', 'clean_sheets', 'creativity', 'element',
       'fixture', 'goals_conceded', 'goals_scored', 'ict_index', 'influence',
       'kickoff_time', 'minutes', 'opponent_team', 'own_goals',
       'penalties_missed', 'penalties_saved', 'red_cards', 'round', 'saves',
       'selected', 'team_a_score', 'team_h_score', 'threat', 'total_points',
       'transfers_balance', 'transfers_in', 'transfers_out', 'value',
       'was_home', 'yellow_cards'],
      dtype='object')

In [89]:
columns_list[483]

Index(['assists', 'bonus', 'bps', 'clean_sheets', 'creativity', 'element',
       'fixture', 'goals_conceded', 'goals_scored', 'ict_index', 'influence',
       'kickoff_time', 'minutes', 'opponent_team', 'own_goals',
       'penalties_missed', 'penalties_saved', 'red_cards', 'round', 'saves',
       'selected', 'team_a_score', 'team_h_score', 'threat', 'total_points',
       'transfers_balance', 'transfers_in', 'transfers_out', 'value',
       'was_home', 'yellow_cards'],
      dtype='object')

In [86]:
x = 0
for i in range(500):
    x = x + 1
    print(columns_list[i][24])

total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points
total_points

IndexError: index 24 is out of bounds for axis 0 with size 0

In [70]:
history_dfs = {}

for key in histories:
    df = pd.DataFrame(histories[key])
    df['id_'] = key
    df['web_name'] = top500.loc[key]['web_name']
    df['team'] = top500['team'].loc[key]
    df['sum_minutes'] = [sum(df['minutes'].iloc[:i+1]) for i in range(len(df.index))] 
    df['next_week_points'] = df['total_points']
    
    history_dfs[key] = df

KeyError: 'total_points'

In [87]:
x

483

In [ ]:
pd.set_option('display.max_columns', None)
history_dfs[191]

In [ ]:
top500histories = pd.concat(history_dfs.values(), ignore_index=True)
top500histories.shape

In [ ]:
top500histories.to_csv('pre_top500histories.csv')

In [ ]:
top500histories.columns

In [ ]:
top500histories.shape[0]